In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Set dataset and output directory paths
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/Wizard_Images"
OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import shutil

# Create new folder for training images (in correct format)
prepared_dir = "/content/wizard_dreambooth_dataset"
os.makedirs(prepared_dir, exist_ok=True)

# Copy your wizard images from Drive
for i, fname in enumerate(os.listdir(DATA_DIR)):
    if fname.lower().endswith((".png", ".jpg", ".jpeg")):
        src = os.path.join(DATA_DIR, fname)
        dst = os.path.join(prepared_dir, f"wizard_{i}.png")
        shutil.copy(src, dst)

print("🧙 Dataset prepared with", len(os.listdir(prepared_dir)), "images")


🧙 Dataset prepared with 5 images


In [3]:
!ls "/content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth"

Wizard_Images  wizard_model_output


In [4]:
# Save prompts for each image
prompts = [
    "A photo of wizard_man in magical attire, reading a spellbook, glowing wand, storybook style.",
    "A photo of wizard_man flying on a broomstick, holding a glowing wand, warm watercolor background.",
    "wizard_man casting a golden spiral spell in a sunlit forest, storybook style.",
    "wizard_man standing with a glowing staff, facing forward, comic watercolor style.",
    "wizard_man walking on a forest path while casting a glowing spell, hand-drawn storybook style."
]

with open(f"{prepared_dir}/captions.txt", "w") as f:
    for i, caption in enumerate(prompts):
        f.write(f"wizard_{i}.png\t{caption}\n")

print("✅ Captions saved.")


✅ Captions saved.


In [20]:
%pip install huggingface_hub==0.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.48.0 requires huggingface-hub<1.0,>=0.24.0, but you have huggingface-hub 0.20.3 which is incompatible.
accelerate 1.2.1 requires huggingface-hub>=0.21.0, but you have huggingface-hub 0.20.3 which is incompatible.
autotrain-advanced 0.8.36 requires huggingface-hub==0.27.0, but you have huggingface-hub 0.20.3 which is incompatible.
datasets 3.2.0 requires huggingface-hub>=0.23.0, but you have huggingface-hub 0.20.3 which is incompatible.
peft 0.14.0 requires huggingface-hub>=0.25.0, but you have huggingface-hub 0.20.3 which is incompa

In [6]:
%pip install -q diffusers==0.27.2 transformers accelerate peft safetensors
%pip install -q bitsandbytes
%pip install huggingface_hub==0.20.3


  Using cached huggingface_hub-0.20.3-py3-none-any.whl.metadata (12 kB)
Using cached huggingface_hub-0.20.3-py3-none-any.whl (330 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.30.2
    Uninstalling huggingface-hub-0.30.2:
      Successfully uninstalled huggingface-hub-0.30.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.48.0 requires huggingface-hub<1.0,>=0.24.0, but you have huggingface-hub 0.20.3 which is incompatible.
accelerate 1.2.1 requires huggingface-hub>=0.21.0, but you have huggingface-hub 0.20.3 which is incompatible.
autotrain-advanced 0.8.36 requires huggingface-hub==0.27.0, but you have huggingface-hub 0.20.3 which is incompatible.
datasets 3.2.0 requires huggingface-hub>=0.23.0, but you have huggingface-hub 0.20.3 which is incompatible.
peft 0.14.0 requires huggingface-hub>=0.25.0,

In [1]:
%pip install diffusers==0.27.2 transformers==4.39.3 huggingface-hub==0.27.0 peft==0.10.0 accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 849.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [8]:
!git clone https://github.com/huggingface/diffusers.git
%cd diffusers
!pip install -e .


Cloning into 'diffusers'...
remote: Enumerating objects: 89831, done.
remote: Counting objects: 100% (1548/1548), done.
remote: Compressing objects: 100% (1155/1155), done.
remote: Total 89831 (delta 826), reused 397 (delta 389), pack-reused 88283 (from 4)
Receiving objects: 100% (89831/89831), 67.77 MiB | 15.97 MiB/s, done.
Resolving deltas: 100% (65423/65423), done.
/content/diffusers
Obtaining file:///content/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.34.0.dev0-0.editable-py3-none-any.whl size=11367 sha256=fda639a86a56d8aa7e85ddc87283e752c4a59ed682480ed19f2eb46a5071b26c
  Stored in directory: /tmp/pip-ephem-wheel-cache-vb1sq8k6/wheels/30/15/ca/ab6e88c89d6ba7047b3f155894c6c346e7cf06067fd132ae62
Success

In [7]:
%pip install -U diffusers==0.27.2 transformers==4.39.3 huggingface-hub==0.27.0 peft==0.10.0 accelerate bitsandbytes xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
%cd diffusers

!accelerate launch examples/dreambooth/train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --instance_data_dir="/content/wizard_dreambooth_dataset" \
  --output_dir="/content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output" \
  --instance_prompt="A photo of wizard_man" \
  --resolution=1024 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --max_train_steps=300 \
  --checkpointing_steps=100 \
  --seed=42 \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --enable_xformers_memory_efficient_attention \
  --train_text_encoder

[Errno 2] No such file or directory: 'diffusers'
/content/diffusers
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-04-20 16:48:18.026364: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745167698.048781    3031 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745167698.055559    3031 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already be

In [6]:
import torch
print("CUDA available:", torch.cuda.is_available())


CUDA available: True


In [10]:
import os
if os.path.exists("/content/wizard_dreambooth_dataset/captions.txt"):
    os.rename(
        "/content/wizard_dreambooth_dataset/captions.txt",
        "/content/wizard_dreambooth_captions.txt"
    )
    print("✅ captions.txt moved safely.")


✅ captions.txt moved safely.


In [3]:
import os

output_dir = "/content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output"

for root, dirs, files in os.walk(output_dir):
    for file in files:
        print(os.path.join(root, file))


/content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/pytorch_lora_weights.safetensors
/content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/logs/dreambooth-lora-sd-xl/events.out.tfevents.1745167716.c734b3c0085c.3031.0
/content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/logs/dreambooth-lora-sd-xl/1745167716.068609/events.out.tfevents.1745167716.c734b3c0085c.3031.1
/content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/logs/dreambooth-lora-sd-xl/1745167716.0748281/hparams.yml
/content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/checkpoint-100/pytorch_lora_weights.safetensors
/content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/checkpoint-100/optimizer.bin
/content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/checkpoint-100/scheduler.bin
/content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/checkpoint-100/scaler.pt
/content/dri

In [4]:
!zip -r /content/wizard_model_output.zip "/content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output"

  adding: content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/ (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/logs/ (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/logs/dreambooth-lora-sd-xl/ (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/logs/dreambooth-lora-sd-xl/events.out.tfevents.1745167716.c734b3c0085c.3031.0 (deflated 68%)
  adding: content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/logs/dreambooth-lora-sd-xl/1745167716.068609/ (stored 0%)
  adding: content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/logs/dreambooth-lora-sd-xl/1745167716.068609/events.out.tfevents.1745167716.c734b3c0085c.3031.1 (deflated 56%)
  adding: content/drive/MyDrive/Colab Notebooks/NLP_DreamBooth/wizard_model_output/logs/dreambooth-lora-sd-xl/1745167716.0748281/ (stored 0%)
  adding: content/drive/MyD

In [5]:
from google.colab import files
files.download("/content/wizard_model_output.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# This restarts the kernel completely and clears memory + cache
import os
os.kill(os.getpid(), 9)
